# CTE Limitations

- CTEs can only be used in the current query scope
    - Cannot be referenced after the final SELECT statement
- Problematic if you need to reuse your virtual tables for different purposes
- Dependent virtual tables cannot be referenced individually making debugging more difficult

# Temp Tables Introduction

- Temp tables are only limited by the current session
- Basic syntax uses SELECT INTO #
- Disadvantage is once a temp table is created, you cannot run the entire script again
    - Adding DROP TABLE alleviates issue
- When to use temp tables over CTEs
    - When you need to reference virtual tables in multiple outputs
    - When you need to join large datasets in virtual tables
    - When you need a script instead of a query

In [1]:
USE AW2019;

DROP TABLE IF EXISTS #Sales
SELECT
    OrderDate,
    OrderMonth  = DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1),
    TotalDue,
    OrderRank   = ROW_NUMBER() OVER (PARTITION BY DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1) ORDER BY TotalDue DESC)
INTO #Sales
FROM Sales.SalesOrderHeader


DROP TABLE IF EXISTS #AvgSalesMinusTop10
SELECT
    OrderMonth,
    TotalSales = SUM(TotalDue)
INTO #AvgSalesMinusTop10
FROM #Sales
WHERE OrderRank > 10
GROUP BY OrderMonth


DROP TABLE IF EXISTS #Purchases
SELECT 
    OrderDate,
    OrderMonth  = DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1),
    TotalDue,
    OrderRank   = ROW_NUMBER() OVER (PARTITION BY DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1) ORDER BY TotalDue DESC)
INTO #Purchases
FROM .Purchasing.PurchaseOrderHeader


DROP TABLE IF EXISTS #AvgPurchasesMinusTop10
SELECT
    OrderMonth,
    TotalPurchases = SUM(TotalDue)
INTO #AvgPurchasesMinusTop10
FROM #Purchases
WHERE OrderRank > 10
GROUP BY OrderMonth

----------------------------------------
SELECT TOP 10
    S.OrderMonth,
    S.TotalSales,
    P.TotalPurchases
FROM #AvgSalesMinusTop10 AS S
	JOIN #AvgPurchasesMinusTop10 AS P
		ON S.OrderMonth = P.OrderMonth
ORDER BY S.OrderMonth

(31465 rows affected)

(38 rows affected)

(4012 rows affected)

(29 rows affected)

(10 rows affected)

Total execution time: 00:00:00.602

OrderMonth,TotalSales,TotalPurchases
2011-12-01,1019635.6747,7254.3006
2012-01-01,3622013.9215,220767.0679
2012-02-01,1141791.6116,7610.834
2012-03-01,2441839.1531,218226.7469
2012-04-01,1341386.2938,2496.2083
2012-05-01,2259194.0397,5744.3167
2012-06-01,3527254.7224,107628.3985
2012-07-01,2560587.9382,234.7418
2012-08-01,1534278.5579,14737.7394
2012-09-01,2903851.4579,3485.3792


# CREATE and INSERT

- More control of the table being created and data being inserted
- Create data structure in advance, load data with INSERT
- Appropriate for more complex scenarios

In [2]:
USE AW2019;

DROP TABLE IF EXISTS #Sales
CREATE TABLE #Sales (
    OrderDate   DATE,
    OrderMonth  DATE,
    TotalDue    MONEY,
    OrderRank   INT
)

INSERT INTO #Sales (
    OrderDate,
    OrderMonth,
    TotalDue,
    OrderRank
)

SELECT
    OrderDate,
    OrderMonth  = DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1),
    TotalDue,
    OrderRank   = ROW_NUMBER() OVER (PARTITION BY DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1) ORDER BY TotalDue DESC)
FROM Sales.SalesOrderHeader;


DROP TABLE IF EXISTS #AvgSalesMinusTop10
CREATE TABLE #AvgSalesMinusTop10 (
    OrderMonth  DATE,
    TotalSales  MONEY
)

INSERT INTO #AvgSalesMinusTop10 (
    OrderMonth,
    TotalSales
)

SELECT
    OrderMonth,
    TotalSales = SUM(TotalDue)
FROM #Sales
WHERE OrderRank > 10
GROUP BY OrderMonth;


DROP TABLE IF EXISTS #Purchases
CREATE TABLE #Purchases (
    OrderDate   DATE,
    OrderMonth  DATE,
    TotalDue    MONEY,
    OrderRank   INT
)

INSERT INTO #Purchases (
    OrderDate,
    OrderMonth,
    TOtalDue,
    OrderRank
)

SELECT 
    OrderDate,
    OrderMonth  = DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1),
    TotalDue,
    OrderRank   = ROW_NUMBER() OVER (PARTITION BY DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1) ORDER BY TotalDue DESC)
FROM .Purchasing.PurchaseOrderHeader;


DROP TABLE IF EXISTS #AvgPurchasesMinusTop10
CREATE TABLE #AvgPurchasesMinusTop10 (
    OrderMonth      DATE,
    TotalPurchases  MONEY
)

INSERT INTO #AvgPurchasesMinusTop10 (
    OrderMonth,
    TotalPurchases
)

SELECT
    OrderMonth,
    TotalPurchases = SUM(TotalDue)
FROM #Purchases
WHERE OrderRank > 10
GROUP BY OrderMonth;

----------------------------------------
SELECT TOP 10
    S.OrderMonth,
    S.TotalSales,
    P.TotalPurchases
FROM #AvgSalesMinusTop10 AS S
	JOIN #AvgPurchasesMinusTop10 AS P
		ON S.OrderMonth = P.OrderMonth
ORDER BY S.OrderMonth

(31465 rows affected)

(38 rows affected)

(4012 rows affected)

(29 rows affected)

(10 rows affected)

Total execution time: 00:00:00.413

OrderMonth,TotalSales,TotalPurchases
2011-12-01,1019635.6747,7254.3006
2012-01-01,3622013.9215,220767.0679
2012-02-01,1141791.6116,7610.834
2012-03-01,2441839.1531,218226.7469
2012-04-01,1341386.2938,2496.2083
2012-05-01,2259194.0397,5744.3167
2012-06-01,3527254.7224,107628.3985
2012-07-01,2560587.9382,234.7418
2012-08-01,1534278.5579,14737.7394
2012-09-01,2903851.4579,3485.3792


# TRUNCATE

- Allows us to clear table while keeping structure intact
- Useful for situations when we want to reuse temp tables and reduce clutter in our code

In [3]:
USE AW2019;

-- Create Orders Table
DROP TABLE IF EXISTS #Orders
CREATE TABLE #Orders (
    OrderDate   DATE,
    OrderMonth  DATE,
    TotalDue    MONEY,
    OrderRank   INT
)

-- Insert Sales Data
INSERT INTO #Orders (
    OrderDate,
    OrderMonth,
    TotalDue,
    OrderRank
)

SELECT
    OrderDate,
    OrderMonth  = DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1),
    TotalDue,
    OrderRank   = ROW_NUMBER() OVER (PARTITION BY DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1) ORDER BY TotalDue DESC)
FROM Sales.SalesOrderHeader;


-- Create AvgOrdersMinusTop10 Table
DROP TABLE IF EXISTS #AvgOrdersMinusTop10
CREATE TABLE #AvgOrdersMinusTop10 (
    OrderMonth  DATE,
    OrderType   VARCHAR(32),
    TotalDue    MONEY
)

-- Insert Sales Data
INSERT INTO #AvgOrdersMinusTop10 (
    OrderMonth,
    OrderType,
    TotalDue
)

SELECT
    OrderMonth,
    OrderType   = 'Sales',
    TotalDue    = SUM(TotalDue)
FROM #Orders
WHERE OrderRank > 10
GROUP BY OrderMonth;


-- Empty Orders Table
TRUNCATE TABLE #Orders;


-- Insert Purchase Data
INSERT INTO #Orders (
    OrderDate,
    OrderMonth,
    TotalDue,
    OrderRank
)

SELECT
    OrderDate,
    OrderMonth  = DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1),
    TotalDue,
    OrderRank   = ROW_NUMBER() OVER (PARTITION BY DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1) ORDER BY TotalDue DESC)
FROM Purchasing.PurchaseOrderHeader;


-- Insert Purchase Data
INSERT INTO #AvgOrdersMinusTop10 (
    OrderMonth,
    OrderType,
    TotalDue
)

SELECT
    OrderMonth,
    OrderType   = 'Purchase',
    TotalDue    = SUM(TotalDue)
FROM #Orders
WHERE OrderRank > 10
GROUP BY OrderMonth;

----------------------------------------
SELECT TOP 10
    A.OrderMonth,
    TotalSales      = A.TotalDue,
    TotalPurchases  = B.TotalDue
FROM #AvgOrdersMinusTop10 AS A
    JOIN #AvgOrdersMinusTop10 AS B
        ON A.OrderMonth = B.OrderMonth
        AND B.OrderType = 'Purchase'
WHERE A.OrderType = 'Sales'
ORDER BY A.OrderMonth;

(31465 rows affected)

(38 rows affected)

(4012 rows affected)

(29 rows affected)

(10 rows affected)

Total execution time: 00:00:00.216

OrderMonth,TotalSales,TotalPurchases
2011-12-01,1019635.6747,7254.3006
2012-01-01,3622013.9215,220767.0679
2012-02-01,1141791.6116,7610.834
2012-03-01,2441839.1531,218226.7469
2012-04-01,1341386.2938,2496.2083
2012-05-01,2259194.0397,5744.3167
2012-06-01,3527254.7224,107628.3985
2012-07-01,2560587.9382,234.7418
2012-08-01,1534278.5579,14737.7394
2012-09-01,2903851.4579,3485.3792


# UPDATE

- Change data in an exisiting table in-place
- Oppurtunity for query optimization
- Allows us to build layers of logic without resorting to complex logic
    - Build derived fields and logic based on other derived fields and logic

In [4]:
USE AW2019;

-- Create Sales Orders Table
DROP TABLE IF EXISTS #SalesOrders
CREATE TABLE #SalesOrders (
    SalesOrderID        INT,
    OrderDate           DATE,
    TaxAmt              MONEY,
    Freight             MONEY,
    TotalDue            MONEY,
    TaxFreightPercent   FLOAT,
    TaxFreightBucket    VARCHAR(32),
    OrderAmtBucket      VARCHAR(32),
    OrderCategory       VARCHAR(32),
    OrderSubcategory    VARCHAR(32)
)

-- Insert Sales Order Data
INSERT INTO #SalesOrders (
    SalesOrderID,
    OrderDate,
    TaxAmt,
    Freight,
    TotalDue,
    OrderCategory
)

SELECT
    SalesOrderID,
    OrderDate,
    TaxAmt,
    Freight,
    TotalDue,
    OrderCategory = 'Non-Holiday Order'
FROM Sales.SalesOrderHeader
WHERE YEAR (OrderDate) = 2013;


-- Update Sales Order Data
UPDATE #SalesOrders
SET
    TaxFreightPercent   =   (TaxAmt + Freight)/TotalDue,
    OrderAmtBucket      =   CASE
                                WHEN TotalDue < 100 THEN 'Small'
                                WHEN TotalDue < 1000 THEN 'Medium'
                                ELSE 'Large'
	                        END;    

-- Update Sales Order Data Dependent on Previous Update
UPDATE #SalesOrders
SET    
    TaxFreightBucket =      CASE
                                WHEN TaxFreightPercent < 0.1 THEN 'Small'
                                WHEN TaxFreightPercent < 0.2 THEN 'Medium'
                                ELSE 'Large'
                            END;

-- Update Order Category
UPDATE #SalesOrders
SET OrderCategory = 'Holiday'
FROM #SalesOrders
WHERE DATEPART (QUARTER, OrderDate) = 4

-- Update Order Subcategory
UPDATE #SalesOrders
SET OrderSubcategory = CONCAT (OrderCategory, ' - ', OrderAmtBucket)
FROM #SalesOrders

----------------------------------------
SELECT TOP 10 *
FROM #SalesOrders

(14182 rows affected)

(14182 rows affected)

(14182 rows affected)

(6121 rows affected)

(14182 rows affected)

(10 rows affected)

Total execution time: 00:00:00.672

SalesOrderID,OrderDate,TaxAmt,Freight,TotalDue,TaxFreightPercent,TaxFreightBucket,OrderAmtBucket,OrderCategory,OrderSubcategory
54413,2013-08-14,3.9184,1.2245,54.1229,0.095,Small,Small,Non-Holiday Order,Non-Holiday Order - Small
54414,2013-08-14,2.9832,0.9323,41.2055,0.095,Small,Small,Non-Holiday Order,Non-Holiday Order - Small
54415,2013-08-14,4.7168,1.474,65.1508,0.095,Small,Small,Non-Holiday Order,Non-Holiday Order - Small
54416,2013-08-14,144.676,45.2113,1998.3373,0.095,Small,Large,Non-Holiday Order,Non-Holiday Order - Large
54417,2013-08-14,190.3576,59.4868,2629.3144,0.095,Small,Large,Non-Holiday Order,Non-Holiday Order - Large
54418,2013-08-14,62.1064,19.4083,857.8447,0.095,Small,Medium,Non-Holiday Order,Non-Holiday Order - Medium
54419,2013-08-14,62.4656,19.5205,862.8061,0.095,Small,Medium,Non-Holiday Order,Non-Holiday Order - Medium
54420,2013-08-14,43.1992,13.4998,596.689,0.095,Small,Medium,Non-Holiday Order,Non-Holiday Order - Medium
54421,2013-08-14,48.6368,15.199,671.7958,0.095,Small,Medium,Non-Holiday Order,Non-Holiday Order - Medium
54422,2013-08-14,197.3056,61.658,2725.2836,0.095,Small,Large,Non-Holiday Order,Non-Holiday Order - Large


# DELETE

- Alows us to selectively remove rows from our data by applying criteria
- Must be careful not to accidentally delete records

In [5]:
USE AW2019;

-- Create Sales Table
DROP TABLE IF EXISTS #Sales
SELECT 
    OrderDate,
	OrderMonth = DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1),
    TotalDue,
	OrderRank = ROW_NUMBER() OVER (PARTITION BY DATEFROMPARTS (YEAR (OrderDate), MONTH (OrderDate), 1) ORDER BY TotalDue DESC)
INTO #Sales
FROM Sales.SalesOrderHeader;

-- Delete Records w/ Criteria
DELETE FROM #Sales WHERE OrderRank > 10;

-- Delete All Records (Similar to TRUNCATE)
-- DELETE FROM #Sales

----------------------------------------
SELECT TOP 10*
FROM #Sales;

(31465 rows affected)

(31085 rows affected)

(10 rows affected)

Total execution time: 00:00:00.124

OrderDate,OrderMonth,TotalDue,OrderRank
2013-09-30 00:00:00.000,2013-09-01,137721.3102,1
2013-09-30 00:00:00.000,2013-09-01,126230.418,2
2013-09-30 00:00:00.000,2013-09-01,124249.4919,3
2013-09-30 00:00:00.000,2013-09-01,105403.8246,4
2013-09-30 00:00:00.000,2013-09-01,105120.5963,5
2013-09-30 00:00:00.000,2013-09-01,97248.3525,6
2013-09-30 00:00:00.000,2013-09-01,89661.1827,7
2013-09-30 00:00:00.000,2013-09-01,86193.0846,8
2013-09-30 00:00:00.000,2013-09-01,84763.4917,9
2013-09-30 00:00:00.000,2013-09-01,83688.5469,10
